In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} .cm-s-ipython span.cm-comment {color: #87CEFA; font-style: italic;}</style>"))

In [4]:
pd.options.display.max_columns = 1000

In [5]:
pd.options.display.max_rows = 1000

In [6]:
cust = pd.read_csv("CUST_DATA.csv", encoding="utf-16")
claim = pd.read_csv("CLAIM_DATA.csv", encoding="utf-16")

결측치 확인

In [7]:
# import missingno
# missingno.matrix(cust)
# plt.show()

In [8]:
# import missingno
# missingno.matrix(claim)
# plt.show()

# CUST_DATA

## 나이

In [9]:
cust['AGE'] = cust['AGE'].map(lambda x: int(x//10))

In [10]:
cust['LTBN_CHLD_AGE'] = cust['LTBN_CHLD_AGE'].map(lambda x: x//10)

## 직업

In [11]:
cust.drop("OCCP_GRP_2", axis=1, inplace=True)

In [12]:
cust['OCCP_GRP_1'] = cust['OCCP_GRP_1'].map(lambda x:str(x)[2:])

In [13]:
cust.drop("MATE_OCCP_GRP_2", axis=1, inplace=True)

In [14]:
cust['MATE_OCCP_GRP_1'] = cust['MATE_OCCP_GRP_1'].map(lambda x: str(x)[2:])

## Date 관련 변수 삭제

In [15]:
from datetime import datetime
today_y = str(datetime.today().year)
today_m = str(datetime.today().month)
today = today_y + today_m

In [16]:
def calc_month(a, b):
    try:
        a = str(a)
        a_y = int(a[:4])
        a_m = int(a[4:6])

        b = str(b)
        b_y = int(b[:4])
        b_m = int(b[4:6])
    
        return (a_y - b_y) * 12 + (a_m - b_m) + 1
    except:
        return 0

In [17]:
max_ym = list()
for i in range(len(cust)):
    max_ym.append(calc_month(cust.MAX_PAYM_YM[i], cust.CUST_RGST[i]))

In [18]:
cust["CUST_RGST"] = cust.CUST_RGST.map(lambda x: calc_month(today, x))
cust["MAX_PAYM_YM"] = max_ym

In [19]:
# cust.drop(['CUST_RGST', 'MAX_PAYM_YM'], axis=1, inplace=True)

In [20]:
cust.to_csv("CUST_DATA_1-1.csv", encoding="utf-8-sig", index=False)

In [21]:
# import missingno
# missingno.bar(cust)
# plt.show()

# 결측치 처리

신용등급

In [22]:
cust.drop(['MINCRDT', 'MAXCRDT'], axis=1, inplace=True)

개인소득 

* 직업별 평균 소득으로 채움

In [23]:
occp_age_cust = cust.pivot_table(index=["AGE", "OCCP_GRP_1"],
                                 values="CUST_INCM", aggfunc="mean")

In [24]:
# occp_age_cust.head()

In [25]:
def fill_income(row):
    try:
        avg_income = occp_age_cust["CUST_INCM"][row.AGE][row.OCCP_GRP_1]
    except:
        avg_income = 0
    return avg_income

In [26]:
cust["CUST_INCM"][cust.CUST_INCM.isnull()] = cust[cust.CUST_INCM.isnull()].apply(fill_income, axis=1)

C:\Users\COM\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


가구 소득

In [27]:
avg_income_by_job = cust.pivot_table(index=["OCCP_GRP_1"],
                                     values=["JPBASE_HSHD_INCM"],
                                     aggfunc="mean")

In [28]:
def fill_jpbase_incm(row):
    try:
        avg_jpbase = jpbase_df["JPBASE_HSHD_INCM"][row.OCCP_GRP_1]
    except:
        avg_jpbase = 0
    return avg_jpbase

In [29]:
cust['JPBASE_HSHD_INCM'][cust.JPBASE_HSHD_INCM.isnull()] = cust[cust['JPBASE_HSHD_INCM'].isnull()].apply(fill_jpbase_incm, axis=1)

C:\Users\COM\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
# cust.to_csv("CUST_DATA_1-2.csv", encoding='utf-8-sig', index=False)

기타 변수

* RESI_TYPE_CODE, CTPR: 최빈값

In [31]:
# cust = pd.read_csv("CUST_DATA_1-2.csv", encoding='utf-8-sig')

In [32]:
# cust

In [33]:
resi_mean = cust.groupby("RESI_TYPE_CODE").mean()

In [34]:
# resi_mean

In [35]:
# cust["RESI_TYPE_CODE"].value_counts()

In [36]:
cust["RESI_TYPE_CODE"].fillna(20, inplace=True)

In [37]:
def na2mean(x):
    if x.RESI_COST==0:
        x.RESI_COST = resi_mean.loc[x['RESI_TYPE_CODE']]['RESI_COST']
    return x

In [38]:
cust = cust.apply(na2mean, axis=1)

In [39]:
# cust

In [40]:
# len(cust.loc[cust.CTPR.isnull()])

In [41]:
# cust["CTPR"].value_counts()

In [42]:
cust["CTPR"].fillna("경기", inplace=True)

* WEDD_YN: N

In [43]:
cust["WEDD_YN"].fillna("N", inplace=True)

In [44]:
cust.head()

,CUST_ID,DIVIDED_SET,SIU_CUST_YN,SEX,AGE,RESI_COST,RESI_TYPE_CODE,FP_CAREER,CUST_RGST,CTPR,OCCP_GRP_1,TOTALPREM,WEDD_YN,MATE_OCCP_GRP_1,CHLD_CNT,LTBN_CHLD_AGE,MAX_PAYM_YM,MAX_PRM,CUST_INCM,RCBASE_HSHD_INCM,JPBASE_HSHD_INCM
0,1,1,N,2,4,21111.000000,20.0,N,249,충북,사무직,146980441.0,Y,사무직,2.0,1.0,110,319718.0,4879.000000,10094,11337.0
1,2,1,N,1,5,40000.000000,20.0,N,249,서울,사무직,94600109.0,Y,주부,2.0,1.0,15,341341.0,6509.000000,9143,6509.0
2,3,1,N,1,6,22133.007305,20.0,N,249,서울,서비스,18501269.0,N,n,0.0,0.0,44,131300.0,4180.000000,0,4180.0
3,4,1,N,2,6,12861.000000,40.0,Y,249,경기,자영업,317223657.0,N,n,0.0,0.0,132,1493184.0,4813.837838,4270,5914.0
4,5,1,N,2,5,22133.007305,20.0,Y,249,광주,자영업,10506072.0,Y,사무직,3.0,1.0,106,166760.0,3894.000000,0,8885.0


In [45]:
# len(cust.loc[cust.WEDD_YN.isnull()]), len(cust.loc[cust.MATE_OCCP_GRP_1.isnull()]), len(cust.loc[cust.LTBN_CHLD_AGE.isnull()]), 

* LBTN_CHLD_AGE, CHLD_CNT, TOTALPREM, MAX_PRM: 0

In [46]:
def fillna_zero(cust, col):
    return cust[col].fillna(0, inplace=True)

In [47]:
fillna_zero(cust, "LTBN_CHLD_AGE")
fillna_zero(cust, "CHLD_CNT")
fillna_zero(cust, "TOTALPREM")
fillna_zero(cust, "MAX_PRM")

* FP_CAREER: 분석에 유의미하지 않다고 판단, 삭제

In [48]:
cust.drop("FP_CAREER", axis=1, inplace=True)

## 원-핫 인코딩

In [49]:
category = ["SEX", "RESI_TYPE_CODE", "CTPR", "OCCP_GRP_1", "WEDD_YN", "MATE_OCCP_GRP_1"]
cust = pd.get_dummies(cust, columns=category)

In [ ]:
cust.to_csv("CUST_DATA_after_dummy.csv", index=False, encoding="utf-8-sig")

# CLAIM_DATA

* 결측치가 너무 많은 변수, 날짜변수 삭제

In [50]:
# 결측치가 지나치게 많음(그냥 흰기둥수준)
drops = ["SELF_CHAM", "NON_PAY", "TAMT_SFCA", "PATT_CHRG_TOTA", "DSCT_AMT", "COUNT_TRMT_ITEM", "DCAF_CMPS_XCPA"]
claim.drop(drops, axis=1, inplace=True)

In [51]:
# 입원 기간은 따로 변수 있음(VLID_HOSP_OTDA)
# 얼마나 자주 입원하는지를 뽑아낼 수 있으면 좋을 것 같긴 한데..좀 어렵다
date_drops = ["HOSP_OTPA_STDT", "HOSP_OTPA_ENDT"]
claim.drop(date_drops, axis=1, inplace=True)

## HOSP_DAYS 
각 고객별 평균 입원 일수

In [52]:
claim_df = claim.pivot_table(index='CUST_ID', values='VLID_HOSP_OTDA', aggfunc='mean')
claim_df.reset_index(inplace=True)
cust = pd.merge(cust, claim_df, how='left', on='CUST_ID')
cust.rename(columns={"VLID_HOSP_OTDA":"HOSP_DAYS"}, inplace=True)

## HEED_HOSP
각 고객별 유의병원 출입여부

In [53]:
id_heed = claim[claim['HEED_HOSP_YN'] == 'Y']['CUST_ID'].unique()

In [54]:
cust['HEED_HOSP'] = np.nan
cust['HEED_HOSP'][cust['CUST_ID'].isin(id_heed)] = 1 # 방문한 적이 있다면 1
cust['HEED_HOSP'].fillna(0, inplace=True) # 나머지는 nan으로 초기화되어 결측치로 취급되고 있음. 0으로 채우기

C:\Users\COM\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## CLAIM_COUNT
각 고객별 청구횟수

In [55]:
# claim_count = claim.loc[:, ["CUST_ID", "POLY_NO"]]
# claim_count_g = claim_count.groupby(claim_count.CUST_ID).count()
# cust["CLAIM_COUNT"] = claim_count_g
# cust.CLAIM_COUNT.fillna(1, inplace=True)

In [56]:
c_count = claim.pivot_table(index='CUST_ID', values='POLY_NO', aggfunc='count').reset_index()
c_count.columns = ['CUST_ID', 'CLAIM_COUNT']
cust = pd.merge(cust, c_count, on='CUST_ID', how='left')

## 의사별 사기비율

In [57]:
# 의사별 사기수/전체 청구수
data = pd.merge(claim, cust[['CUST_ID', 'SIU_CUST_YN']], how='left', on='CUST_ID')
doc_all = data.pivot_table(index=["CHME_LICE_NO"], values="POLY_NO", aggfunc="count").reset_index()
doc_YN = data.pivot_table(index=["CHME_LICE_NO", "SIU_CUST_YN"], values="POLY_NO", aggfunc='count').reset_index()
doc_YN = doc_YN[doc_YN['SIU_CUST_YN']=='Y']
doc_all = pd.merge(doc_all, doc_YN, how='left', on='CHME_LICE_NO')

doc_all["DOC_SIU_RATIO"] = doc_all["POLY_NO_y"]/doc_all["POLY_NO_y"]
doc_all.fillna(0, inplace=True)
doc_fake = doc_all[['CHME_LICE_NO', 'DOC_SIU_RATIO']]

In [58]:
doc_fake.head()

,CHME_LICE_NO,DOC_SIU_RATIO
0,6,0.0
1,7,0.0
2,8,1.0
3,9,0.0
4,10,0.0


In [59]:
# CUST_ID 별 의사사기비율 max
df_lice = claim.pivot_table(index=["CUST_ID", "CHME_LICE_NO"], values="POLY_NO", aggfunc="count").reset_index()
df_lice = pd.merge(data, doc_fake, how='left', on='CHME_LICE_NO')
idlice_ratio = df_lice.pivot_table(index='CUST_ID', values='DOC_SIU_RATIO', aggfunc='max').reset_index()

In [60]:
cust = pd.merge(cust, idlice_ratio, how='left', on='CUST_ID')

## HOSP_SIU_RATIO 
병원별 사기비율

In [61]:
# 병원별 사기수/전체 청구수
hosp_all = data.pivot_table(index=['HOSP_CODE'], values='POLY_NO', aggfunc='count').reset_index()
hosp_YN = data.pivot_table(index=['HOSP_CODE', 'SIU_CUST_YN'], values='POLY_NO', aggfunc='count').reset_index()
hosp_YN = hosp_YN[hosp_YN["SIU_CUST_YN"]=='Y']
hosp_all = pd.merge(hosp_all, hosp_YN, how="left", on="HOSP_CODE")

hosp_all["HOSP_SIU_RATIO"] = hosp_all["POLY_NO_y"] / hosp_all["POLY_NO_x"]
hosp_all.fillna(0, inplace=True)
hosp_fake = hosp_all[["HOSP_CODE", "HOSP_SIU_RATIO"]]

In [62]:
# CUST_ID별 병원 사기비율 max
df_hosp = claim.pivot_table(index=["CUST_ID", "HOSP_CODE"], values="POLY_NO", aggfunc="count").reset_index()
df_hosp = pd.merge(data, hosp_fake, how='left', on='HOSP_CODE')

In [63]:
idhosp_ratio = df_hosp.pivot_table(index="CUST_ID", values="HOSP_SIU_RATIO", aggfunc="max").reset_index()

In [64]:
cust = pd.merge(cust, idhosp_ratio, how='left', on='CUST_ID')

## HOSP_DOC_SIU
의사 사기 비율 * 병원 사기 비율

In [65]:
cust["HOSP_DOC_SIU"] = cust["DOC_SIU_RATIO"] * cust["HOSP_SIU_RATIO"]

## ACCI_RATIO_N 
각 ACCI_DVSN(1,2,3) / 청구횟수

In [66]:
acci_claim = claim.pivot_table(index=["CUST_ID", "ACCI_DVSN"], values="POLY_NO", aggfunc="count").unstack(1).reset_index()
acci_claim = acci_claim.values
acci_claim = pd.DataFrame(acci_claim)
acci_claim.fillna(0, inplace=True)
acci_claim.columns = ["CUST_ID", "ACCI_RATIO_1", "ACCI_RATIO_2", "ACCI_RATIO_3"]

In [67]:
cust = pd.merge(cust, acci_claim, on="CUST_ID", how="left")
cust["ACCI_RATIO_1"] = cust["ACCI_RATIO_1"]/cust["CLAIM_COUNT"]
cust["ACCI_RATIO_2"] = cust["ACCI_RATIO_2"]/cust["CLAIM_COUNT"]
cust["ACCI_RATIO_3"] = cust["ACCI_RATIO_3"]/cust["CLAIM_COUNT"]

In [68]:
cust.ACCI_RATIO_1.fillna(0, inplace=True)
cust.ACCI_RATIO_2.fillna(0, inplace=True)
cust.ACCI_RATIO_3.fillna(0, inplace=True)

## 사고원인(ACCI_DVSN), 청구코드(DMND_RESN_CODE)

In [69]:
claim_acci = claim.loc[:, ["CUST_ID", "ACCI_DVSN", "DMND_RESN_CODE"]]
claim_acci["value"] = 1
cust_claim_df = claim_acci.pivot_table(index=["CUST_ID"],
                                       columns=["ACCI_DVSN", "DMND_RESN_CODE"],
                                       values=["value"],
                                       aggfunc="count", fill_value=0)

In [70]:
cust_claim_df = cust_claim_df.reset_index(level=["CUST_ID"], col_level=1)
cust_claim_df.columns = cust_claim_df.columns.droplevel(level=0)

In [71]:
cust_claim_df.columsn = ["_".join([str(col) for col in cols]) for cols in cust_claim_df.columns]

C:\Users\COM\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [72]:
cust_claim_df = cust_claim_df.rename(columns={"CUST_ID_":"CUST_ID"})

In [73]:
cust = pd.merge(cust, cust_claim_df, how="left", on="CUST_ID")

C:\Users\COM\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\COM\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


# 추가코드

수정 필요

In [ ]:
cust.to_csv("CUST_DATA_2-1.csv", encoding="utf-8-sig", index=False)

In [ ]:
cust

In [ ]:
import math

In [ ]:
def cpr_nan(x):
    if math.isnan(x['NON_PAY_RATIO']):
        x['NON_PAY_RATIO'] = cpr_mean.loc[x['CUST_ID']]['NON_PAY_RATIO']

In [ ]:
claim_pay_ratio = claim.loc[:, ["CUST_ID", "NON_PAY_RATIO"]]
cpr_mean = claim_pay_ratio.groupby(claim_pay_ratio.CUST_ID).mean()
cust["PAY_RATIO_AVG"] = cpr_mean

In [ ]:
cust.PAY_RATIO_AVG.fillna(0, inplace=True)

In [ ]:
cust

In [ ]:
cust = pd.merge(cust, claim_pay_ratio, how='left', on='CUST_ID')

In [ ]:
cust

In [ ]:
cust = pd.read_csv("CUST_DATA_2-1.csv", encoding="utf-8-sig")

# 최종

## 정규화 

In [74]:
scaling_features = cust[["RESI_COST", "TOTALPREM", "MAX_PRM", "CUST_INCM", "RCBASE_HSHD_INCM", "JPBASE_HSHD_INCM"]]

In [75]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_nd = scaler.fit_transform(scaling_features)
scaled_df = pd.DataFrame(scaled_nd, columns=scaling_features.columns)

In [76]:
cust.drop(["RESI_COST", "TOTALPREM", "MAX_PRM", "CUST_INCM", "RCBASE_HSHD_INCM", "JPBASE_HSHD_INCM"], axis=1, inplace=True) # 정규화 전 변수 삭제

In [77]:
cust = pd.concat([cust, scaled_df], axis=1)

In [78]:
def yn_10(x):
    if x=='Y': return 1
    elif x=='N': return 0
    else: pass

In [79]:
cust["SIU_CUST_YN"] = cust["SIU_CUST_YN"].apply(yn_10)

In [ ]:
cust.to_csv("CUST_DATA_results.csv", index=False, encoding="utf-8-sig")

In [80]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(10,8))
# sns.heatmap(cust.corr(), vmin=-1, vmax=1, annot=False, cmap="BrBG")
# plt.show()

In [81]:
cust

,CUST_ID,DIVIDED_SET,SIU_CUST_YN,AGE,CUST_RGST,CHLD_CNT,LTBN_CHLD_AGE,MAX_PAYM_YM,SEX_1,SEX_2,RESI_TYPE_CODE_11.0,RESI_TYPE_CODE_12.0,RESI_TYPE_CODE_13.0,RESI_TYPE_CODE_20.0,RESI_TYPE_CODE_30.0,RESI_TYPE_CODE_40.0,RESI_TYPE_CODE_50.0,RESI_TYPE_CODE_60.0,RESI_TYPE_CODE_70.0,RESI_TYPE_CODE_99.0,CTPR_강원,CTPR_경기,CTPR_경남,CTPR_경북,CTPR_광주,CTPR_대구,CTPR_대전,CTPR_부산,CTPR_서울,CTPR_세종,CTPR_울산,CTPR_인천,CTPR_전남,CTPR_전북,CTPR_제주,CTPR_충남,CTPR_충북,OCCP_GRP_1_1차산업,OCCP_GRP_1_n,OCCP_GRP_1_기타,OCCP_GRP_1_사무직,OCCP_GRP_1_서비스,OCCP_GRP_1_자영업,OCCP_GRP_1_전문직,OCCP_GRP_1_제조업,OCCP_GRP_1_주부,WEDD_YN_N,WEDD_YN_Y,MATE_OCCP_GRP_1_1차산업,MATE_OCCP_GRP_1_n,MATE_OCCP_GRP_1_기타,MATE_OCCP_GRP_1_사무직,MATE_OCCP_GRP_1_서비스,MATE_OCCP_GRP_1_자영업,MATE_OCCP_GRP_1_전문직,MATE_OCCP_GRP_1_제조업,MATE_OCCP_GRP_1_주부,HOSP_DAYS,HEED_HOSP,CLAIM_COUNT,DOC_SIU_RATIO,HOSP_SIU_RATIO,HOSP_DOC_SIU,ACCI_RATIO_1,ACCI_RATIO_2,ACCI_RATIO_3,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)","(1, 6)","(1, 7)","(1, 9)","(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)","(2, 6)","(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)","(3, 6)","(3, 7)","(3, 9)",RESI_COST,TOTALPREM,MAX_PRM,CUST_INCM,RCBASE_HSHD_INCM,JPBASE_HSHD_INCM
0,1,1,0.0,4,249,2.0,1.0,110,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1.250000,0.0,4,1.0,0.200000,0.200000,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0.267662,2.824554,-0.019169,1.093896,1.733306,2.234250
1,2,1,0.0,5,249,2.0,1.0,15,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,2.666667,0.0,3,1.0,0.428571,0.428571,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1.575092,1.638565,-0.000197,1.828832,1.423768,0.521100
2,3,1,0.0,6,249,0.0,0.0,44,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,16.000000,0.0,1,0.0,0.000000,0.000000,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.338401,-0.084456,-0.184485,0.778730,-1.552162,-0.305314
3,4,1,0.0,6,249,0.0,0.0,132,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0.000000,0.0,9,1.0,0.000000,0.000000,1.0,0.0,0.0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.303374,6.679180,1.010419,1.064516,-0.162332,0.309973
4,5,1,0.0,5,249,3.0,1.0,106,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,25.000000,0.0,1,1.0,0.354183,0.354183,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.338401,-0.265482,-0.153373,0.649779,-1.552162,1.364192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22395,20048,2,NaN,3,112,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,18.000000,0.0,2,0.0,0.050000,0.000000,1.0,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.802356,-0.503359,-0.299686,0.395933,-0.373247,-0.606570
22396,21483,1,0.0,3,205,0.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,3.700000,0.0,10,1.0,0.055288,0.055288,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0,2,0,0,0,1.017484,-0.503359,-0.299686,-1.105951,1.494724,1.386191
22397,21940,1,0.0,2,186,0.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3.400000,0.0,5,1.0,0.444444,0.444444,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,1.988453,-0.503359,-0.299686,-1.105951,2.198102,2.286411
22398,20356,1,0.0,1,205,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2.000000,0.0,2,0.0,0.019031,0.000000,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0.729059,-0.503359,-0.29968

cust.isnull().sum()

# 모델링

In [82]:
train = cust[cust["DIVIDED_SET"]==1].reset_index(drop=True)
test = cust[cust["DIVIDED_SET"]==2].reset_index(drop=True)

In [83]:
train.drop("DIVIDED_SET", axis=1, inplace=True)
test.drop("DIVIDED_SET", axis=1, inplace=True)

In [84]:
train_X = train.drop(['CUST_ID', 'SIU_CUST_YN'], axis=1)
train_y = train["SIU_CUST_YN"]

In [85]:
test_X = test.drop(["CUST_ID", "SIU_CUST_YN"], axis=1)

In [93]:
y_val = test.SIU_CUST_YN

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_features=16, random_state=42)
rf_model.fit(train_X, train_y)

In [ ]:
y_pred_rf = rf_model.predict(test_X)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_val, y_pred_rf)

# XGBoost

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(max_depth=10, learning_rate=0.01, n_estimators=100)
xgb_model.fit(train_X, train_y)

In [ ]:
y_pred = xgb_model.predict(test_X)
pd.crosstab(y_val, y_pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_val, y_pred)

# SVM

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(random_state=42)
svm_clf.fit(train_X, train_y)

In [ ]:
y_pred_svm = svm_clf.predict(test_X)
pd.crosstab(y_val, y_pred_svm)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_val, y_pred_svm)

In [87]:
!pip install lightgbm

# LightGBM

In [95]:
x = pd.read_csv("answer.csv")
y_val = x.YN

In [88]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=1, num_leaves=76, max_depth=11)
lgbm_model.fit(train_X.to_numpy(), train_y)
y_pred_lgbm = lgbm_model.predict(test_X)
#f1_score(y_val, y_pred_lgbm)

In [89]:
result = pd.DataFrame(data=test.CUST_ID, columns=['CUST_ID'])
result['YN'] = y_pred_lgbm.astype(int)
result.sort_values(by="CUST_ID", inplace=True)
result.reset_index(drop=True, inplace=True)

In [90]:
result.to_csv("CUST_DATA_results_5조.csv", index=False, encoding='utf-8-sig')

In [96]:
from sklearn.metrics import f1_score
f1_score(y_val, y_pred_lgbm)

0.15944055944055943

In [ ]:
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=1, num_leaves=76, max_depth=11)
lgbm_model.fit(train_X.to_numpy(), train_y)
y_pred_lgbm = lgbm_model.predict(test_X)
f1_score(y_val, y_pred_lgbm)

In [ ]:
pd.crosstab(y_val, y_pred_lgbm)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_val, y_pred_lgbm))

# 앙상블

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_model = VotingClassifier(estimators=[('rf', rf_model), ('xgb', xgb_model), ('lgb', lgbm_model)],
                                voting='hard')
voting_model.fit(train_X.to_numpy(), train_y)

In [ ]:
y_pred = voting_model.predict(test_X.to_numpy())
pd.crosstab(y_val, y_pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_val, y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))

# 오버샘플링

In [ ]:
sample = pd.read_csv("CUST_DATA_전처리완료.csv", encoding='utf-8-sig')

In [ ]:
train = sample[sample['DIVIDED_SET']==1].reset_index(drop=True)
test = sample[sample['DIVIDED_SET']==2].reset_index(drop=True)

In [ ]:
train.drop("DIVIDED_SET", axis=1, inplace=True)
test.drop("DIVIDED_SET", axis=1, inplace=True)

In [ ]:
train_X = train.drop(["CUST_ID", "SIU_CUST_YN"], axis=1)
train_y = train["SIU_CUST_YN"]

In [ ]:
test_X = test.drop(["CUST_ID", "SIU_CUST_YN"], axis=1)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_resampled, y_resampled = sm.fit_sample(train_X, (train_y))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.3, shuffle=True)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_features=16)
rf_model.fit(X_train, y_train)

In [ ]:
y_pred = rf_model.predict(X_val)

In [ ]:
pd.crosstab(y_val, y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))

In [ ]:
# 계층
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3, stratify=train_y)

In [ ]:
sm = SMOTE()
X_train_r, y_train_r = sm.fit_sample(X_train, y_train)
rf_model = RandomForestClassifier(n_estimators=100, max_features=2)
rf_model.fit(X_train_r, y_train_r)

In [ ]:
pred_r = rf_model.predict(X_test)

In [ ]:
pd.crosstab(y_test, pred_r)

In [ ]:
print(classification_report(y_test, pred_r))